In [1]:
####    Get the Paths to data:
import os 
from storyTools import loadStory,splitStoryEntity

analysisName = "TEST_res100_dgv5_bll500_blb75_smw10_smL2000_swG50_trO10000_trB5000"


workingDir = os.getcwd();
path = os.path.split(workingDir)[0] + "\\data" #data path
savePath = os.path.join(workingDir,analysisName,'individuals')
subjects = [f for f in os.listdir(path) if  not os.path.isfile(os.path.join(path, f))] # get all the filenames

subjects
[Story1,Story1_raw] = loadStory("story3.txt")
Story1_bundle = splitStoryEntity(Story1_raw)
karolinaStory = " ".join(Story1_bundle['firstEntity'])
janekStory = " ".join(Story1_bundle['secondEntity'])
#### Get the Original Story Data: 

In [2]:
# import os
# import wave
# import sys
# import json
# from vosk import Model, KaldiRecognizer

# # Your input .wav file

# for subj in subjects:
#     currentSubj = os.path.join(path,subj, 'Recalls', 'story1.wav')

#     # Open wave file
#     wf = wave.open(currentSubj, "rb")

#     # Check audio format
#     if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
#         print("Audio file must be WAV format mono PCM.")
#         sys.exit(1)

#     # Load Vosk model
#     model = Model(lang="pl")
#     rec = KaldiRecognizer(model, wf.getframerate())

#     # Accumulate final text here
#     full_text = ""

#     while True:
#         data = wf.readframes(4000)
#         if len(data) == 0:
#             break
#         if rec.AcceptWaveform(data):
#             result = json.loads(rec.Result())
#             full_text += result.get("text", "") + " "

#     # Final result
#     final_result = json.loads(rec.FinalResult())
#     full_text += final_result.get("text", "")

#     # Print or return the complete transcription
#     print(full_text)


In [ ]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import wave
from faster_whisper import WhisperModel
from storyTools import cosineSim
import nltk
import sys
nltk.download('punkt_tab')

janekScore = [];
karolinaScore = [];
allScore = [];

for i, subj in enumerate(subjects):
    currentSubj = os.path.join(path,subj, 'Recalls', 'story1.wav')

    if i < 2:
        continue

    # Open wave file
    wf = wave.open(currentSubj, "rb")

    # Check audio format
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        print("Audio file must be WAV format mono PCM.")
        sys.exit(1)

    

    # Load Whisper model (can use "base", "small", "medium", etc.)
    model = WhisperModel("medium", device="cuda")  # or "cuda" if you have a GPU

    segments, _ = model.transcribe(currentSubj, language="pl", beam_size=20) # włąściwy 
    originalText = [s.text for s in segments]
    print("You said: ", originalText)

    # Save Recall Text: C
    with open(os.path.join(savePath,subj,'recall.txt'), "w") as text_file:
        text_file.write(" ".join(originalText))


    
    janekScore.append(cosineSim(" ".join(originalText),janekStory))
    karolinaScore.append(cosineSim(" ".join(originalText),karolinaStory))
    allScore.append(cosineSim(" ".join(originalText),"".join(Story1)))

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\barak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


You said:  [' Więc nagrywam opowiadanie o historii, którą właśnie usłyszałam.', ' Była ona o rodzeństwie, właśnie Karolinie i jej braci Janku, którzy generalnie byli bardzo z biednej rodziny,', ' mieszkali na poddaszu Młyna i doskwierała właśnie im bieda.', ' Karolina chciała, właśnie podobało jej się dotychczasowe życie.', ' Chciała właśnie bogactwa życia na salonach i po prostu większego docenienia wśród innych.', ' Z kolei Janek akceptował życie, którym właśnie żyje.', ' Nie potrzebował takiej sławy uznania, a dla niego najbardziej liczyła się pomoc dla innych, wsparcie,', ' żeby po prostu czuł się spełniony pracą, którą miał.', ' I pewnego dnia Karolina przechadzając się zauważyła kobietę, która leżała na ziemi,', ' wyglądała na osobę taką ubogą, bezdomną w jakimś starym płaszczu, lecz przeszła obok', ' z myślą, że może pomóc, ale jednak się obawiała, że nie ma czasu na zajmowanie się życiem innych.', ' Zaś wracając, zobaczyła Janka, który zainteresował się kobietą, chciał jej pomó

: 